# Сравнение методов линейного поиска, backtracking method

Про реализацию и теорОбоснование `Backtracking method` — см. в первых частях.

## Что хотим от line search метода

Мы уже собрали целый зоопарк методов линейного поиска:
- БинПоиск по градиенту (с остановкой по достижению малого значения градиента и/или размера ворот $r - l$, с опциональным обрубанием по количеству итераций)
- Золотое сечение (по количеству итераций)
- Фибоначчи (по количеству итераций)
- Метод, базирующийся на условиях Вольфе (aka backtracking method)

Здесь нужно проанализировать, какие `line searcher`-ы _лучше_ себя показывают. Что от такого алгоритма требуется?

1. Конечно же, хорошо сходиться
2. Произвести _достаточно_ точный поиск, чтобы заЭксплойтить это направление поиска
3. Но не переусердствовать: не слишком долго искать, чтобы сэкономить время

Пункт 1 очевиден и не настолько интересен.
А вот между 2. и 3. налицо трейд-офф по поводу того, насколько быстро надо останавливаться. Для каждой конкретной функции ответ на этот вопрос разный — и хороший line search метод — тот, который хорошо умеет его давать.
Причём давать для самых разных паттернов ландшафта, масштабирований и т.д.

## Как производить оценку

Раз уж так важна устойчивость и догадливость метода, заведём набор _интересных_ функций и попробуем запустить методы на них (метод проходит все функции с одинаковыми гиперпараметрами — приближено к реальности).

Функции:
- квадратичная (какое-то/какие-то число(а) обусловленности, размерность(и))
- умножение на разные константы (маленькую, большую)/масштабирование аргументов
- непредсказуемый рельеф (всё ещё выпуклая, но где-то сильно убывает, где-то — _не очень_…)

Теперь — как оценивать результат. В целом — чем меньше действий, тем лучше. В реальной жизни обычно речь идёт о процессорном времени, но здесь измерять время — это лишняя возня, потеря точности и, главное, bias за счёт того, что у тестовых функций (к примеру, квадратичной) может быть _странное_ отношение времён вычисления функции и градиента. Это затрудняет оценку самого метода линейного поиска, а также не соответствует реальности. Поэтому будем считать, что целевая функция (чем меньше, тем лучше) в данном случае $\operatorname{computations}(f) + \operatorname{computations}(\nabla f)$.

Ожидаем, что методы с фиксированным количеством итераций будут хорошо себя проявлять только для конкретных функций фиксированного уровня сложности, а для других — либо делать избыточную работу внутри каждой итерации, либо плохо отрабатывать очередное направление → потребуется много внешних итераций, иногда даже вообще не сходиться.

А про методы с остановкой по фиксированному $\varepsilon$ ожидаем, что он будет плохо устойчив к масштабированию функции/градиента.

Реальность соответствует ожиданиям: backtracking method умеет с одной стороны вовремя остановиться, с другой — чётко отрабатывает (→ не расходится), когда это нужно.

In [73]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 10)
%load_ext autoreload
%autoreload 2

from core.gradient_descent import *
from core.visualizer import *
from core.optimizer_evaluator import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
test_linear_search(fixed_step_search(0.1), True)

Quadratic : f called 42 times, f' called 41 times, total score: 83
Trigonometric : f called 101 times, f' called 100 times, total score: 201
GD diverged at Rosenbrock (got vector norm 67215268.07617551 after 3 steps), such a pity…
GD diverged at 100-dimensional quadratic with k = 100 (got vector norm 2.279063127909417e+18 after 16 steps), such a pity…


In [75]:
test_linear_search(fixed_step_search(1), True)

Quadratic : f called 6 times, f' called 5 times, total score: 11
Trigonometric : f called 3 times, f' called 2 times, total score: 5
GD diverged at Rosenbrock (got vector norm 697073038160.035 after 3 steps), such a pity…
GD diverged at 100-dimensional quadratic with k = 100 (got vector norm 9.909305564164147e+19 after 10 steps), such a pity…


In [76]:
test_linear_search(wolfe_conditions_search(0.05, 0.9), True)

Quadratic : f called 68 times, f' called 30 times, total score: 98
Trigonometric : f called 364 times, f' called 152 times, total score: 516
Rosenbrock : f called 262 times, f' called 60 times, total score: 322
100-dimensional quadratic with k = 100 : f called 1793 times, f' called 507 times, total score: 2300


In [77]:
test_linear_search(fibonacci_search(15), True)

Quadratic : f called 617 times, f' called 14 times, total score: 631
Trigonometric : f called 1865 times, f' called 42 times, total score: 1907
GD diverged at Rosenbrock (got vector norm 647125619911.2137 after 6 steps), such a pity…
100-dimensional quadratic with k = 100 : f called 7833 times, f' called 178 times, total score: 8011


In [78]:
test_linear_search(fibonacci_search(30), True)

Quadratic : f called 891 times, f' called 10 times, total score: 901
Trigonometric : f called 911 times, f' called 10 times, total score: 921
Rosenbrock : f called 1781 times, f' called 20 times, total score: 1801
100-dimensional quadratic with k = 100 : f called 17178 times, f' called 193 times, total score: 17371


In [79]:
test_linear_search(golden_ratio_search, True)

Quadratic : f called 511 times, f' called 10 times, total score: 521
Trigonometric : f called 550 times, f' called 10 times, total score: 560
Rosenbrock : f called 1021 times, f' called 20 times, total score: 1041
100-dimensional quadratic with k = 100 : f called 9334 times, f' called 183 times, total score: 9517
